In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def create_model(base_model):
    base_model.trainable = True
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    prediction_layer = tf.keras.layers.Dense(9, activation='softmax')(global_average_layer)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=["accuracy"])
    return model

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min",
    restore_best_weights=True,
)

In [ ]:
batch_size = 32
epochs = 3
def fit_model(model):
    history = model.fit(train_ds,
                        batch_size=batch_size, 
                        epochs=epochs,
                        callbacks=[callback], 
                        validation_data=val_ds)
    return history
IMG_SHAPE = (224, 224, 3)

In [ ]:
base_model1 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model2 = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model3 = tf.keras.applications.Xception(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [ ]:
model1 = create_model(base_model1)
model1.summary
model2 = create_model(base_model2)
model2.summary
model3 = create_model(base_model3)
model3.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f1733d6a3d0>>

In [ ]:
model1.input

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>

In [ ]:
history1 = fit_model(model1)
model1.save('models/model1.h5')
history2 = fit_model(model2)
model2.save('models/model2.h5')
history3 = fit_model(model3)
model3.save('models/model3.h5')

Epoch 1/3
157/157 [==============================] - 1135s 7s/step - loss: 1.0273 - accuracy: 0.6620 - val_loss: 1.1625 - val_accuracy: 0.6125
Epoch 2/3
157/157 [==============================] - 996s 6s/step - loss: 0.2763 - accuracy: 0.9174 - val_loss: 0.7290 - val_accuracy: 0.7814
Epoch 3/3
157/157 [==============================] - 1000s 6s/step - loss: 0.1117 - accuracy: 0.9726 - val_loss: 0.6086 - val_accuracy: 0.8215


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/3
157/157 [==============================] - 2764s 18s/step - loss: 0.8408 - accuracy: 0.7142 - val_loss: 0.4018 - val_accuracy: 0.8759
Epoch 2/3
157/157 [==============================] - 2933s 19s/step - loss: 0.1946 - accuracy: 0.9412 - val_loss: 0.3243 - val_accuracy: 0.8999
Epoch 3/3
157/157 [==============================] - 2902s 18s/step - loss: 0.0600 - accuracy: 0.9856 - val_loss: 0.3119 - val_accuracy: 0.9103
Epoch 1/3
157/157 [==============================] - 5481s 35s/step - loss: 0.9728 - accuracy: 0.6914 - val_loss: 0.3325 - val_accuracy: 0.9039
Epoch 2/3
157/157 [==============================] - 5372s 34s/step - loss: 0.2075 - accuracy: 0.9348 - val_loss: 0.2523 - val_accuracy: 0.9231
Epoch 3/3
157/157 [==============================] - 5324s 34s/step - loss: 0.0656 - accuracy: 0.9826 - val_loss: 0.2730 - val_accuracy: 0.9239


In [ ]:
path = '/content/test/ImageID_USRB8QNG.jpg'

img = tf.keras.utils.load_img(
    path, target_size=(img_height, img_height)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

prediction1 = model1.predict(img_array)
score1 = tf.nn.softmax(prediction1[0])

prediction2 = model2.predict(img_array)
score2 = tf.nn.softmax(prediction2[0])

prediction3 = model3.predict(img_array)
score3 = tf.nn.softmax(prediction3[0])

print(
    "Model1 : This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score1)], 100 * np.max(score1))
)
print(
    "Model2 : This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score2)], 100 * np.max(score2))
)
print(
    "Model3 : This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score3)], 100 * np.max(score3))
)

Model1 : This image most likely belongs to Temple with a 25.17 percent confidence.
Model2 : This image most likely belongs to Temple with a 25.36 percent confidence.
Model3 : This image most likely belongs to Temple with a 25.17 percent confidence.


In [ ]:
ScoreM1=[]
ScoreM2=[]
ScoreM3=[]

ClassM1=[]
ClassM2=[]
ClassM3=[]

for i in range(len(test)):
  path = '/content/test/'+test['img_IDS'][i]+'.jpg'
  img = tf.keras.utils.load_img(
      path, target_size=(img_height, img_height))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) 

  prediction1 = model1.predict(img_array)
  score1 = tf.nn.softmax(prediction1[0])

  ScoreM1.append(100 * np.max(score1))
  ClassM1.append(class_names[np.argmax(score1)])

  prediction2 = model2.predict(img_array)
  score2 = tf.nn.softmax(prediction2[0])

  ScoreM2.append(100 * np.max(score2))
  ClassM2.append(class_names[np.argmax(score2)])

  prediction3 = model3.predict(img_array)
  score3 = tf.nn.softmax(prediction3[0])

  ScoreM3.append(100 * np.max(score3))
  ClassM3.append(class_names[np.argmax(score3)])

test['ScoreM1']=ScoreM1
test['ScoreM2']=ScoreM2
test['ScoreM3']=ScoreM3

test['ClassM1']=ClassM1
test['ClassM2']=ClassM2
test['ClassM3']=ClassM3

In [ ]:
test.to_csv('/content/Sub.csv')

In [ ]:
test.head()

,img_IDS,ScoreM1,ScoreM2,ScoreM3,ClassM1,ClassM2,ClassM3
0,ImageID_USRB8QNG,25.167266,25.356334,25.169438,Temple,Temple,Temple
1,ImageID_SZ8D1ZJI,18.550976,25.359789,25.307578,Enough,Enough,Enough
2,ImageID_4OJO2F8J,24.175632,25.296560,25.187659,Mosque,Mosque,Mosque
3,ImageID_IEE4XV0B,24.253249,25.349993,25.305197,Mosque,Mosque,Mosque
4,ImageID_BP0O0WZ9,25.146490,24.933249,22.388220,You,You,You


In [ ]:
test.describe()

,ScoreM1,ScoreM2,ScoreM3
count,2679.000000,2679.000000,2679.000000
mean,22.737515,24.346815,24.288585
std,3.494986,2.466881,2.507300
min,12.387562,12.129620,11.211013
25%,20.937052,25.068800,24.959814
50%,24.682479,25.317559,25.262591
75%,25.291337,25.351015,25.324242
max,25.361040,25.361156,25.360674


In [ ]:
Diff=test
Diff.head()

,img_IDS,ScoreM1,ScoreM2,ScoreM3,ClassM1,ClassM2,ClassM3
1,ImageID_SZ8D1ZJI,18.550976,25.359789,25.307578,Enough,Enough,Enough
2,ImageID_4OJO2F8J,24.175632,25.296560,25.187659,Mosque,Mosque,Mosque
3,ImageID_IEE4XV0B,24.253249,25.349993,25.305197,Mosque,Mosque,Mosque
4,ImageID_BP0O0WZ9,25.146490,24.933249,22.388220,You,You,You
5,ImageID_M0V58H3K,25.218588,25.343397,25.329655,Church,Church,Church


In [ ]:
Diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2678 entries, 1 to 2678
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   img_IDS  2678 non-null   object 
 1   ScoreM1  2678 non-null   float64
 2   ScoreM2  2678 non-null   float64
 3   ScoreM3  2678 non-null   float64
 4   ClassM1  2678 non-null   object 
 5   ClassM2  2678 non-null   object 
 6   ClassM3  2678 non-null   object 
dtypes: float64(3), object(4)
memory usage: 231.9+ KB


In [ ]:
Diff.head()

,Image_ID,Church,Enough/Satisfied,Friend,Love,Me,Mosque,Seat,Temple,You
1,ImageID_SZ8D1ZJI,0,1,0,0,0,0,0,0,0
2,ImageID_4OJO2F8J,0,0,0,0,0,1,0,0,0
3,ImageID_IEE4XV0B,0,0,0,0,0,1,0,0,0
4,ImageID_BP0O0WZ9,0,0,0,0,0,0,0,0,1
5,ImageID_M0V58H3K,1,0,0,0,0,0,0,0,0
